In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind,shapiro,f,mannwhitneyu, ttest_rel,f_oneway,pearsonr,bartlett
import seaborn as sns
from sklearn.linear_model import LinearRegression
from itertools import combinations
import os

## **REGRESSIONS**

### Regression linéaire

In [ ]:
for i in matrice.columns : 
    for j in matrice.columns : 
        if i != j : 
            x = matrice[i]
            y = matrice[j]
            
            lin = pd.concat([x, y], axis=1).dropna()

            x = lin.iloc[:, 0]
            y = lin.iloc[:, 1]

            X=np.array(x).reshape(-1, 1)
            model = LinearRegression()
            model.fit(X, y)
            t=np.linspace(min(x),max(x),100)

            R2 = model.score(X, y)

            
            def ft (t):
                return model.coef_[0]*t+model.intercept_
                
            if R2 > 0.05 :
                if not (os.path.exists("{x.name} x {y.name}.csv") or os.path.exists("{y.name} x {x.name}.csv") ):
                    plt.figure()
                    plt.scatter(x,y)
                    plt.plot(t,ft(t),'firebrick',label=f"R2={R2:.4f}")
                    plt.xlabel(f"{x.name}")
                    plt.ylabel(f"{y.name}")
                    plt.legend() 
                    plt.title(f"regression linéaire de {x.name} x {y.name}")
                    plt.show()



### Regression multiple

In [ ]:

import statsmodels.api as sm

df_vm["Sexe_num"] = df_vm["Sexe"].map({"F": 0, "M": 1})
covm = pd.concat([matrice, df_vm["Age"], df_vm["Sexe_num"]], axis=1)
covariables = ['leptin', "Age", "Sexe_num"]
resultats_ajustes = []

for var in matrice.columns:
    X = sm.add_constant(covm[covariables])
    y = matrice[var]
    model = sm.OLS(y, X, missing='drop').fit()
    resid_y = model.resid

    print(model.summary()) 

    resultats_ajustes.append({
    'Variable': var,
    f'Beta (Leptine)': model.params['leptin'],
    f'p-value (Leptine)': model.pvalues['leptin'],
    'intercept' : model.params["const"],
    'R²': model.rsquared,
    'R2 adj' : model.rsquared_adj
    })

    x = df_vm["Leptine"]
    sns.scatterplot(x=x, y=y, alpha=0.6)
    t=np.linspace(x.min(), x.max(), 100)
    plt.plot(t,(resultats_ajustes[-1][f"Beta (Leptine)"] * t + resultats_ajustes[-1]["intercept"]),'r',label=f"R2 = {resultats_ajustes[-1]['R2 adj']} \npvalue = {resultats_ajustes[-1]['p-value (Leptine)']:.6f}")
    plt.legend()
    plt.title(f"regression multiple de leptine sur {var}")
    plt.show()
    
df_resultats_ajustes = pd.DataFrame(resultats_ajustes)
print(df_resultats_ajustes[df_resultats_ajustes["p-value (Leptine)"]<0.05])



### regression partielle

In [ ]:
import statsmodels.api as sm
df_vm["Sexe"] = df_vm["Sexe"].replace({"F": 0, "M": 1})
df_vm["Diabète"] = df_vm["TypeDiabetes_status"].replace({"non" : 0,"IG" : 1,"type2" : 2})

for x, y in combinations(matrice.columns, 2): #pour faire des combinaisons de toutes les colonnes (x<->y, et pas x->y + y->x)

    if x == y:
        continue 
    covars = ["Age", "Sexe"]
    MTX = pd.concat([df_vm["Age"],df_vm["Sexe"],matrice[x],matrice[y]],axis=1).dropna()
    X_cov = sm.add_constant(MTX[covars])
    model_x = sm.OLS(MTX[x], X_cov, missing='drop').fit()
    resid_x = model_x.resid
    model_y = sm.OLS(MTX[y], X_cov, missing='drop').fit()
    resid_y = model_y.resid
    resid_df = pd.DataFrame({"resid_x" : resid_x, "resid_y" : resid_y})
    X_resid = sm.add_constant(resid_df['resid_x'])
    model_resid = sm.OLS(resid_df['resid_y'], X_resid).fit()
    r_squared_adj = model_resid.rsquared_adj
    r,p = pearsonr(resid_df['resid_x'],resid_df['resid_y'])

    if p < 0.05 : 
        plt.figure(figsize=(6,5))
        sns.regplot(data=resid_df,x='resid_x', y='resid_y',ci=95, line_kws={'color' : 'red'},label=f"r = {r:.2f}, p = {p:.3e} \n R² adj = {r_squared_adj:.3f}")
        plt.xlabel(f"{x} (adjusted)")
        plt.ylabel(f"{y} (adjusted)")
        plt.title(f"correlation partielle ajustée par le sexe, l'âge\n {x} et {y}")
        plt.legend()
        plt.tight_layout()
        plt.show()




In [ ]:
pvalues = df_resultats_ajustes["p-value (Leptine)"]
pvalues.index=[df_resultats_ajustes["Variable"]]
pvalues = pvalues.to_frame(name="p-value (Leptine)") #mettre en 2D

plt.figure(figsize=(5, 10))
ax =sns.heatmap(pvalues, annot=True, cmap='coolwarm', fmt=".5f", cbar=True)


significant_mask = pvalues["p-value (Leptine)"] < 0.05
highlight = np.where(significant_mask.values[:, None], 2, 0)  # épaisseur des bordures
for y, (value, sig) in enumerate(zip(pvalues["p-value (Leptine)"], significant_mask)):
    if sig:
        ax.add_patch(plt.Rectangle((0, y), 1, 1, fill=False, edgecolor='black', lw=3))


plt.title("valeurs de pvalues")
plt.tight_layout()
plt.show()

### plot régression partielle

In [ ]:
# code régression partielle 

import statsmodels.api as sm
for var in matrice.columns :
    X = sm.add_constant(covm[covariables])
    y = matrice[var]
    model = sm.OLS(y, X, missing='drop').fit()
    resid_y = model.resid

    df_vm[f"{var}_pred"] = model.fittedvalues
    df_vm["residus"] = model.resid

    plt.figure(figsize=(7, 5))
    sns.scatterplot(x=df_vm["Leptine"], y=y)
    sns.lineplot(x=df_vm["Leptine"], y=df_vm[f"{var}_pred"], color="red", label="Régression ajustée")
    plt.xlabel("Leptine")
    plt.ylabel(f"{var}")
    plt.title(f"Régression ajustée : {var} ~ Leptine + Age + Sexe")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
import statsmodels.api as sm


df_vm["Sexe"] = df_vm["Sexe"].replace({"F": 0, "M": 1})
df_vm["fatmass totale (g)"] = analyse["fatmass totale (g)"]

x = "leptin"
y = "nbr_cell_total"

covars = ["Age", "Sexe", "fatmass totale (g)"]
MTX = pd.concat([df_vm["Age"],df_vm["Sexe"],df_vm["fatmass totale (g)"],matrice[x],matrice[y]],axis=1).dropna()
X_cov = sm.add_constant(MTX[covars])
model_x = sm.OLS(MTX[x], X_cov, missing='drop').fit()
resid_x = model_x.resid
model_y = sm.OLS(MTX[y], X_cov, missing='drop').fit()
resid_y = model_y.resid
resid_df = pd.DataFrame({"resid_x" : resid_x, "resid_y" : resid_y})
X_resid = sm.add_constant(resid_df['resid_x'])
model_resid = sm.OLS(resid_df['resid_y'], X_resid).fit()
r_squared_adj = model_resid.rsquared_adj
r,p = pearsonr(resid_df['resid_x'],resid_df['resid_y'])
plt.figure(figsize=(6,5))
sns.regplot(data=resid_df,x='resid_x', y='resid_y',ci=95, line_kws={'color' : 'red'},label=f"r = {r:.2f}, p = {p:.3e} \n R² adj = {r_squared_adj:.3f}")
plt.xlabel(f"{x} (adjusted)")
plt.ylabel(f"{y} (adjusted)")
plt.title(f"correlation partielle ajustée par le sexe, l'âge et la fatmass (g)\n {x} et {y}")
plt.legend()
plt.tight_layout()
plt.show()




### TEST COMPARAISON DE MOYENNE INDEP/APPARIEES


In [ ]:
#on veut tester : 

# CONDITIONS A REMPLIR : 
te = "indep", "apparié"
type_echantillons = "apparié"

groupe1=analyse["Nb cellules sc"].dropna()
name1="nombre cellules sc total"

groupe2=analyse['Nb cellules om'].dropna()
name2="nombre cellules om total"


In [ ]:

# TEST DE STUDENT
# lin = pd.concat([groupe1, groupe2], axis=1).dropna()

# groupe1 = lin.iloc[:, 0]
# groupe2 = lin.iloc[:, 1]

#### DEBUT DU TEST 
results = []

mean1=np.mean(groupe1)
mean2=np.mean(groupe2)

print(f"On fait un test de comparaison de moyennes {type_echantillons}\nmoyenne de {name1} = {mean1} et moyenne de {name2} = {mean2} \n")

## test de normalité des données
stat1, p1 = shapiro(groupe1)
stat2, p2 = shapiro(groupe2)
#si normalité respectée : 

if p1 > 0.05 and p2 > 0.05:
    print(f"NORMALITE = les groupes {name1} et {name2} ont une distribution normale")

        # Homogénéité des variances
    # var1 = np.var(groupe1, ddof=1)
    # var2 = np.var(groupe2, ddof=1)
    # f_stat = max(var1, var2) / min(var1, var2)
    # df1, df2 = len(groupe1)-1, len(groupe2)-1
    # p_var = 2 * (1 - f.cdf(f_stat, df1, df2))
    b_stat, p_var = bartlett(groupe1, groupe2)   

    if p_var > 0.05:
        print(f"HOMEOSCEDASTICITE : homéoscédasticité respectée (p_var = {p_var:.6f})")

        if type_echantillons == "indep" : 
            t_stat, pval = ttest_ind(groupe1, groupe2,equal_var=True)
            test = "t-test indépendant"
        else : 
            t_stat, pval = ttest_rel(groupe1, groupe2,equal_var=True)
            test = "t-test apparié"
        if pval < 0.05 :
            print(f"TEST : Test de student : YAY la distribution est significativement différente à p-value={pval:.22f}")
        else : 
            print(f"TEST : Test de student : NOO moyennes non différentes avec p-value={pval:.5f}")

    else:
        print(f"HOMEOSCEDASTICITE : homéoscédasticité non respectée (p_var = {p_var:.6f})")
        if type_echantillons == "indep" : 
            t_stat, pval = ttest_ind(groupe1, groupe2,equal_var=False)
            test = "test de Welch indépendant"
        else : 
            t_stat, pval = ttest_rel(groupe1, groupe2,equal_var=False)
            test = "test de Welch apparié"
        
        if pval < 0.05 :
            print(f"TEST : Test non paramétrique (Welch): moyennes sont significativement différentes : p-value = {pval:.20f}")
        else : 
            print(f"TEST : Test non paramétrique (Welch): moyennes NON significativement différentes : p-value = {pval:.5f}")

        # stat, pval = mannwhitneyu(groupe1, groupe2, alternative='two-sided')
        # test = "mannwhitney"
        # if pval<0.05 :
        #     print(f"TEST : Test non paramétrique (mann whitney): moyennes sont significativement différentes : p-value = {pval:.20f}")
        # else : 
        #     print(f"TEST : Test non paramétrique (mann whitney): moyennes NON significativement différentes : p-value = {pval:.5f}")
        
else:
    p_var="pas d'homoscédasticité"
    print(f"NORMALITE Les distributions ne sont pas normales : p-value de {name1}={p1:.8f} et p-value de {name2}={p2:.8f}")
    stat, pval = mannwhitneyu(groupe1, groupe2, alternative='two-sided')
    test = "mannwhitney"
    if pval<0.05 :
        print(f"TEST : Test non paramétrique (mann whitney): moyennes sont significativement différentes : p-value = {pval:.20f}")
    else : 
        print(f"TEST : Test non paramétrique (mann whitney): moyennes NON significativement différentes : p-value = {pval:.5f}")


if pval < 0.05 : 
    conclusion = "test significatif *"
    if pval < 0.01 : 
        conclusion = "test significatif **" 
    if pval < 0.001 : 
        conclusion = "test significatif ***"  
else : 
    conclusion = "test non significatif"

results.append({
            "groupe 1": name1,
            "groupe 2": name2,
            "normalité groupe 1" : p1,
            "normalité groupe 2" : p2,
            "homoscédasticité" : p_var,
            "p_value": pval,
            "test": test,
            "type test" : type_echantillons,
            "conclusion" : conclusion
        })
 

results_df = pd.DataFrame(results)
results_df

# Niveau de significativité
if pval < 0.001:
    sig = '***'
elif pval < 0.01:
    sig = '**'
elif pval < 0.05:
    sig = '*'
else:
    sig = 'ns'

# Plot avec moyenne ± écart-type
# Créer un DataFrame pour le plot
df_plot = pd.DataFrame({
    "valeur": pd.concat([groupe1, groupe2], ignore_index=True),
    "groupe": [name1] * len(groupe1) + [name2] * len(groupe2)
})
plt.figure(figsize=(7, 5))
ax = sns.barplot(data=df_plot, x="groupe", y="valeur", hue="groupe",errorbar="sd", capsize=0.1, palette="pastel", err_kws={"linewidth": 1.5})

y_max = df_plot["valeur"].max()
y_annotation = y_max * 1.05
plt.ylim(0, y_annotation * 1.2)  # marges suffisantes pour le texte
ax.annotate(f"p = {pval:.3e} ({sig})", xy=(0.5, y_annotation), ha='center', fontsize=12)
plt.title(f"comparaison {name1} et {name2}\ntest : {results_df["test"][0]}")
plt.tight_layout()
plt.show()

results_df


In [ ]:

corr = matrice.corr(method='spearman')

plt.figure(figsize=(35, 25))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".3f", square=True)
plt.title("Matrice de corrélation (spearman)")
plt.tight_layout()
plt.show()